In [56]:
import numpy as np
import pandas as pd
from collections import Counter
rng = np.random.default_rng(42)

In [57]:
'''
def load_data(path, *args):
    data = pd.read_csv(path)
    data = data.dropna()
    lst = []
    for header in args:
        if header == "SALARY":
            data[header] = pd.to_numeric(data[header].str.replace(r"[$,]", "", regex=True).str.strip())
        lst.append(np.array(data[header].tolist()))
    return lst

rng = np.random.default_rng(42)
players, points, rebounds, assists, weights = load_data("./nbasalariespoints.csv", "Player", "PS/G▼", "TRB", "AST", "SALARY")
'''
nbadata = pd.read_csv("./nbasalariespoints.csv")
nbadata = nbadata.dropna()
nbadata.keys()
nbadata['SALARY'] = pd.to_numeric(nbadata['SALARY'].str.replace(r"[$,]", "", regex=True).str.strip())

min_minutes = 12
nbadata = nbadata[nbadata['MP'] >= min_minutes] 

salary = nbadata['SALARY']
points = nbadata['PS/G▼']
rebounds = nbadata['TRB']
assists = nbadata['AST']
steals = nbadata['STL']
blocks = nbadata['BLK']
turnovers = nbadata['TOV']
three_pointers = nbadata['3P']
fieldgoals = nbadata['eFG%']
freethrows = nbadata['FT']
fouls = nbadata['PF']
minutes = nbadata['MP']
players = nbadata['Player']
taskdata = pd.DataFrame({
    'Player': players,
    'Salary': salary,
    'Points': 36*points/minutes,
    'Rebounds': 36*rebounds/minutes,
    'Assists': 36*assists/minutes,
    'Steals': 36*steals/minutes,
    'Blocks': 36*blocks/minutes,
}).reset_index(drop=True)


In [ ]:
def initializePopulation(pop_size, team_size):
    return rng.integers(0, len(weights) - 1, (pop_size, team_size), dtype=int)

def calculateValue(individual, obj):
    return np.sum(obj[individual])

def calculateWeight(individual, weights):
    return np.sum(weights[individual])

def calculateFitness(individual, weights, capacity, ideal_point, objectives):
    val_list = []
    total = 0

    for objective in objectives:
        val_list.append(calculateValue(individual, objective))
    
    for i in range(len(ideal_point)):
        diff = ideal_point[i] - val_list[i]
        # Allows for overshooting the point
        if diff > 0:
            total += diff ** 2
    dist = np.sqrt(total)

    # Use 1/dist as fitness
    if dist != 0: fitness = 1/dist
    else: fitness = np.inf

    # Can't choose the same player twice
    if (len(set(individual)) < 15) or (calculateWeight(individual, weights) > capacity):
        fitness = -np.inf
    return fitness

def adjustDistribution(capacity, weights, population, ideal_point, objectives, counter):
    fitness = []
    distribution = np.ones(len(weights)) / len(weights)
    for individual in population:
        fitness.append(calculateFitness(individual, weights, capacity, ideal_point, objectives))
        
    indices = np.argsort(fitness)
    population = population[indices][::-1]
    fitness = np.sort(fitness)[::-1]

    for i in range(len(fitness)):
        if fitness[i] > .75 * fitness[0]:
            counter += Counter(population[i])
        else: break

    for index, count in counter.items():
        distribution[index] = count / len(distribution)
    distribution = distribution / distribution.sum()
    
    return distribution

def Sample(distribution, pop_size, team_size):
    length = len(distribution)
    population = np.zeros((pop_size, team_size), dtype=int)

    for i in range(pop_size):
        population[i] = rng.choice(length, team_size, p = distribution)
    return population

def knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, objectives):
    population = initializePopulation(pop_size, team_size)
    counter = Counter()

    for i in range(training_length):
        distribution = adjustDistribution(capacity, weights, population, ideal_point, objectives, counter)
        population = Sample(distribution, pop_size, team_size)

    indices = np.argsort([val for val in [calculateFitness(individual, weights, capacity, ideal_point, objectives) for individual in population]])
    best_solution = population[indices][-1]
    return best_solution


In [59]:
ideal_point = [300, 140]
capacity = 150_000_000
training_length = 20
pop_size = 1000
team_size = 15
weights = taskdata["Salary"]
objectives = [taskdata["Points"], taskdata["Rebounds"], taskdata["Assists"]]
individual = knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, objectives)
individual = np.sort(individual)
print(individual)
print(taskdata["Player"][individual])
print(calculateWeight(individual, weights))
print(f"Value of objective 1: {calculateValue(individual, objectives[0])}")
print(f"Value of objective 2: {calculateValue(individual, objectives[1])}")
print(f"Value of objective 3: {calculateValue(individual, objectives[2])}")


[  0   2   3   5   6  25  33  35  51 120 127 132 163 193 229]
0          Stephen Curry
2           Kevin Durant
3       DeMarcus Cousins
5         Damian Lillard
6          Anthony Davis
25            Chris Paul
33     LaMarcus Aldridge
35         Jahlil Okafor
51         Zach Randolph
120      Jared Sullinger
127         Nikola Jokic
132          Carl Landry
163            Jon Leuer
193       Tyson Chandler
229        Alexis Ajinca
Name: Player, dtype: object
142861172
Value of objective 1: 316.60442705788904
Value of objective 2: 141.4069422794376
Value of objective 3: 55.42651405445308
